In [1]:
import requests
from datetime import datetime

# Helper function to normalize the date format to dd/mm/yyyy
def normalize_date(date_str):
    # Try to parse the date if it's in 'yyyy-mm-dd' format
    for fmt in ("%Y-%m-%d", "%d/%m/%Y"):
        try:
            # Try to parse the date using the format
            date_obj = datetime.strptime(date_str, fmt)
            return date_obj.strftime("%d/%m/%Y")  # Return the normalized date in dd/mm/yyyy format
        except ValueError:
            continue
    return 'Invalid Date'  # Return if the date format is unknown

url = "https://world.openfoodfacts.org/api/v2/search?fields=code,product_name,brands,quantity,categories,expiration_date&page_size=100"

response = requests.get(url)
data = response.json()

# Loop through products and store the details as variables
for product in data["products"]:
    # Save the details into variables
    product_name = product.get('product_name', 'N/A')
    product_brand = product.get('brands', 'N/A')
    product_quantity = product.get('quantity', 'N/A')
    product_categories = product.get('categories', 'N/A')
    product_expiration_date = product.get('expiration_date', 'N/A')

    # Normalize the expiration date
    normalized_expiration_date = normalize_date(product_expiration_date)

    # If expiration date is invalid, skip the product
    if normalized_expiration_date == 'Invalid Date':
        continue

    # Print the details if expiration date is valid
    print(f"Name: {product_name}")
    print(f"Brand: {product_brand}")
    print(f"Quantity: {product_quantity}")
    print(f"Categories: {product_categories}")
    print(f"Expiration Date: {normalized_expiration_date}")
    print("-" * 100)

Name: Sidi Ali
Brand: Sidi Ali
Quantity: 33 cl
Categories: Beverages and beverages preparations,Beverages,Waters,Spring waters,Mineral waters,Unsweetened beverages,Natural mineral waters
Expiration Date: 24/01/2020
----------------------------------------------------------------------------------------------------
Name: Eau minérale naturelle
Brand: Sidi Ali,Les Eaux Minérales d'oulmès
Quantity: 1,5 L
Categories: Beverages and beverages preparations,Beverages,Waters,Spring waters,Mineral waters,Unsweetened beverages,Natural mineral waters
Expiration Date: 25/02/2019
----------------------------------------------------------------------------------------------------
Name: اكوافينا
Brand: pepsi,PepsiCo
Quantity: 33cl
Categories: Boissons et préparations de boissons,Boissons,Eaux,Eaux de sources,Eaux minérales,Eaux de table
Expiration Date: 24/09/2025
----------------------------------------------------------------------------------------------------
Name: Albeni Bar Cake
Brand: Ülker
Qua

In [4]:
import requests

# Function to get product details using barcode
def get_product_info(barcode):
    url = f"https://world.openfoodfacts.org/api/v2/product/{barcode}.json"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        if "product" in data:
            product_name = data["product"].get("product_name", "N/A")
            categories = data["product"].get("categories", "N/A")
            
            print(f"Product Name: {product_name}")
            print(f"Categories: {categories}")
        else:
            print("Product not found.")
    else:
        print("Error fetching data.")

# Example: Scan a barcode (replace with actual barcode)
barcode = "737628064502"  # Example barcode
get_product_info(barcode)


Product Name: Thai peanut noodle kit includes stir-fry rice noodles & thai peanut seasoning
Categories: Plant-based foods and beverages, Plant-based foods, Cereals and potatoes, Cereals and their products, Pastas, Noodles, Rice Noodles


In [14]:
import cv2
import pytesseract
import speech_recognition as sr
import requests
from pyzbar.pyzbar import decode

# Function to scan barcode using OCR
def scan_barcode(image_path):
    image = cv2.imread(image_path)
    barcodes = decode(image)

    for barcode in barcodes:
        barcode_data = barcode.data.decode("utf-8")
        return barcode_data  # Return first detected barcode

    return None  # No barcode detected

# Function to recognize speech and return barcode number
def recognize_speech():
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Say the barcode number (or say 'stop' to cancel)...")
        recognizer.adjust_for_ambient_noise(source)
        
        try:
            # Start listening with a timeout (stops if no voice is detected in 5 sec)
            audio = recognizer.listen(source, timeout=30, phrase_time_limit=10)

            # Process the recognized text
            barcode_text = recognizer.recognize_google(audio).lower()

            if "stop" in barcode_text:
                print("Recognition stopped by user.")
                return None  # Stop listening but continue the program

            print(f"Recognized Barcode: {barcode_text}")
            return barcode_text.replace(" ", "")  # Remove spaces

        except sr.UnknownValueError:
            print("Could not understand the audio. Skipping...")
        except sr.RequestError:
            print("Speech recognition service error.")
        except sr.WaitTimeoutError:
            print("No speech detected. Moving on...")

    return None  # Return None if recognition failed

# Function to get product details from OpenFoodFacts API
def get_product_info(barcode):
    url = f"https://world.openfoodfacts.org/api/v2/product/{barcode}.json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        if "product" in data:
            product_name = data["product"].get("product_name", "N/A")
            categories = data["product"].get("categories", "N/A")
            return f"Product: {product_name}\nCategories: {categories}"

    return "Product not found."

# Main program logic
def main():
    try:
        while True:
            print("\nChoose an option:\n1. Scan Barcode from Image\n2. Speak Barcode Number\n3. Exit")
            choice = input("Enter choice (1, 2, or 3): ").strip()

            if choice == "1":
                image_path = input("Enter image path: ")  # Provide path to barcode image
                barcode = scan_barcode(image_path)
            elif choice == "2":
                barcode = recognize_speech()  # Stops listening but program continues
            elif choice == "3":
                print("Exiting program.")
                break  # Exit loop
            else:
                print("Invalid choice. Please enter 1, 2, or 3.")
                continue

            if barcode:
                print(f"Barcode: {barcode}")
                product_info = get_product_info(barcode)
                print(product_info)
            else:
                print("No barcode detected or recognized.")

    except KeyboardInterrupt:
        print("\nProgram stopped by user.")

if __name__ == "__main__":
    main()



Choose an option:
1. Scan Barcode from Image
2. Speak Barcode Number
3. Exit


TypeError: cannot unpack non-iterable NoneType object